<a href="https://colab.research.google.com/github/DataChemE/Histopathologic-Cancer-Detection/blob/main/cancer_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!unzip -q /content/drive/MyDrive/histopathologic-cancer-detection.zip

In [2]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())


Num GPUs Available:  1
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 11562608716937781205
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 40427651072
locality {
  bus_id: 1
  links {
  }
}
incarnation: 5655200031550574617
physical_device_desc: "device: 0, name: NVIDIA A100-SXM4-40GB, pci bus id: 0000:00:04.0, compute capability: 8.0"
xla_global_id: 416903419
]


In [3]:
import tensorflow as tf
from tensorflow.keras import layers, models
import pandas as pd
import os
import numpy as np
from sklearn.model_selection import train_test_split
from PIL import Image
import matplotlib.pyplot as plt

# Define paths
BASE_PATH = '/content/drive/MyDrive/histopathologic-cancer-detection'
TRAIN_PATH = os.path.join(BASE_PATH, 'train')
TEST_PATH = os.path.join(BASE_PATH, 'test')
TRAIN_LABELS_PATH = os.path.join(BASE_PATH, 'train_labels.csv')

# Load the training labels
train_labels = pd.read_csv(TRAIN_LABELS_PATH)

def load_image(image_path):
    img = Image.open(image_path)
    img = img.resize((96, 96))  # Resize the image
    img_array = np.array(img) / 255.0  # Normalize to [0, 1]
    return img_array

def create_dataset(image_ids, labels=None, is_train=True):
    def gen():
        for i, id in enumerate(image_ids):
            image_path = os.path.join(TRAIN_PATH if is_train else TEST_PATH, f"{id}.tif")
            img = load_image(image_path)
            if labels is not None:
                yield img, labels.iloc[i]
            else:
                yield img

    if labels is not None:
        output_signature = (
            tf.TensorSpec(shape=(96, 96, 3), dtype=tf.float32),
            tf.TensorSpec(shape=(), dtype=tf.int64)
        )
    else:
        output_signature = tf.TensorSpec(shape=(96, 96, 3), dtype=tf.float32)

    dataset = tf.data.Dataset.from_generator(
        gen,
        output_signature=output_signature
    )

    return dataset

# Split data into train and validation sets
train_ids, val_ids, train_labels, val_labels = train_test_split(
    train_labels['id'], train_labels['label'], test_size=0.2, random_state=12
)

# Configure the datasets for training
BATCH_SIZE = 32
AUTOTUNE = tf.data.AUTOTUNE

# Calculate steps per epoch
train_steps_per_epoch = len(train_ids) // BATCH_SIZE
val_steps_per_epoch = len(val_ids) // BATCH_SIZE

train_dataset = create_dataset(train_ids, train_labels)
train_dataset = train_dataset.shuffle(1000).batch(BATCH_SIZE).repeat().prefetch(AUTOTUNE)

val_dataset = create_dataset(val_ids, val_labels)
val_dataset = val_dataset.batch(BATCH_SIZE).repeat().prefetch(AUTOTUNE)

# Prepare the test dataset
test_ids = [filename.split('.')[0] for filename in os.listdir(TEST_PATH) if filename.endswith('.tif')]
test_dataset = create_dataset(test_ids, labels=None, is_train=False)
test_dataset = test_dataset.batch(BATCH_SIZE).prefetch(AUTOTUNE)

import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Activation, Add, Flatten, Dense, AveragePooling2D, ZeroPadding2D, MaxPooling2D
from tensorflow.keras.models import Model

def identity_block(X, f, filters, stage, block):
    """
    Implementation of the identity block as defined in the original ResNet paper.

    Arguments:
    X -- input tensor of shape (m, height, width, channels)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    stage -- integer, used to name the layers, depending on their position in the network
    block -- string/character, used to name the layers, depending on their position in the network

    Returns:
    X -- output of the identity block, tensor of shape (m, height, width, channels)
    """

    # Retrieve Filters
    F1, F2, F3 = filters

    # Save the input value. You'll need this later to add back to the main path.
    X_shortcut = X

    # First component of main path
    X = Conv2D(filters = F1, kernel_size = (1, 1), strides = (1, 1), padding = 'valid')(X)
    X = BatchNormalization(axis = 3)(X)
    X = Activation('relu')(X)

    # Second component of main path
    X = Conv2D(filters = F2, kernel_size = (f, f), strides = (1, 1), padding = 'same')(X)
    X = BatchNormalization(axis = 3)(X)
    X = Activation('relu')(X)

    # Third component of main path
    X = Conv2D(filters = F3, kernel_size = (1, 1), strides = (1, 1), padding = 'valid')(X)
    X = BatchNormalization(axis = 3)(X)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)

    return X

def convolutional_block(X, f, filters, stage, block, s = 2):
    """
    Implementation of the convolutional block as defined in the original ResNet paper.

    Arguments:
    X -- input tensor of shape (m, height, width, channels)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    stage -- integer, used to name the layers, depending on their position in the network
    block -- string/character, used to name the layers, depending on their position in the network
    s -- Integer, specifying the stride to be used

    Returns:
    X -- output of the convolutional block, tensor of shape (m, height, width, channels)
    """

    # Retrieve Filters
    F1, F2, F3 = filters

    # Save the input value
    X_shortcut = X

    # First component of main path
    X = Conv2D(F1, (1, 1), strides = (s, s), padding = 'valid')(X)
    X = BatchNormalization(axis = 3)(X)
    X = Activation('relu')(X)

    # Second component of main path
    X = Conv2D(F2, (f, f), strides = (1, 1), padding = 'same')(X)
    X = BatchNormalization(axis = 3)(X)
    X = Activation('relu')(X)

    # Third component of main path
    X = Conv2D(F3, (1, 1), strides = (1, 1), padding = 'valid')(X)
    X = BatchNormalization(axis = 3)(X)

    # Shortcut path
    X_shortcut = Conv2D(F3, (1, 1), strides = (s, s), padding = 'valid')(X_shortcut)
    X_shortcut = BatchNormalization(axis = 3)(X_shortcut)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)

    return X

def ResNet50(input_shape = (96, 96, 3), classes = 1):
    """
    Implementation of the popular ResNet50 architecture.

    Arguments:
    input_shape -- shape of the images of the dataset
    classes -- integer, number of classes for the output layer

    Returns:
    model -- a Model() instance in Keras
    """

    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)

    # Zero-Padding
    X = ZeroPadding2D((3, 3))(X_input)

    # Stage 1
    X = Conv2D(64, (7, 7), strides = (2, 2))(X)
    X = BatchNormalization(axis = 3)(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    # Stage 2
    X = convolutional_block(X, f = 3, filters = [64, 64, 256], stage = 2, block='a', s = 1)
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='b')
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='c')

    # Stage 3
    X = convolutional_block(X, f = 3, filters = [128, 128, 512], stage = 3, block='a', s = 2)
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='b')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='c')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='d')

    # Stage 4
    X = convolutional_block(X, f = 3, filters = [256, 256, 1024], stage = 4, block='a', s = 2)
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='b')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='c')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='d')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='e')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='f')

    # Stage 5
    X = convolutional_block(X, f = 3, filters = [512, 512, 2048], stage = 5, block='a', s = 2)
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='b')
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='c')

    # AVGPOOL
    X = AveragePooling2D(pool_size=(2, 2), padding='same')(X)

    # output layer
    X = Flatten()(X)
    X = Dense(classes, activation='sigmoid')(X)

    # Create model
    model = Model(inputs = X_input, outputs = X, name='ResNet50')

    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy', tf.keras.metrics.AUC(name='auc')])

    return model

# Create ResNet50 model
model = ResNet50(input_shape=(96, 96, 3), classes=1)
model.summary()


history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    steps_per_epoch=train_steps_per_epoch,
    validation_steps=val_steps_per_epoch,
    epochs=10,
    callbacks=callbacks
)


# Plotting the training accuracy
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.legend()
plt.title('Accuracy')

plt.subplot(1, 2, 2)
plt.plot(history.history['auc'], label='Train AUC')
plt.plot(history.history['val_auc'], label='Validation AUC')
plt.legend()
plt.title('AUC')

plt.show()

# Evaluate the model on the test set
predictions = model.predict(test_dataset, verbose=1)

# Round the predictions to 0 or 1
rounded_predictions = np.round(predictions).astype(int)

# Create submission DataFrame
submission = pd.DataFrame({
    'id': test_ids,
    'label': rounded_predictions.flatten()
})

# Save submission to CSV
submission.to_csv('submission.csv', index=False)

print("Submission file created: submission.csv")


print(submission.head(10))


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/histopathologic-cancer-detection/train_labels.csv'